# Save and load models

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work.

### Options

There are different ways to save TensorFlow models depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras)—a high-level API to build and train models in TensorFlow. The new, high-level `.keras` format used in this tutorial is recommended for saving Keras objects, as it provides robust, efficient name-based saving that is often easier to debug than low-level or legacy formats.

## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


### Get an example dataset

To demonstrate how to save and load weights, the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) is used. To speed up these runs, the first 1000 examples is used:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


### Define a model

Start by building a simple sequential model:

In [6]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Save checkpoints during training

The `tf.keras.callbacks.ModelCheckpoint` callback allows you to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2334 - sparse_categorical_accuracy: 0.6451
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 23ms/step - loss: 1.1832 - sparse_categorical_accuracy: 0.6610 - val_loss: 0.7589 - val_sparse_categorical_accuracy: 0.7780
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4303 - sparse_categorical_accuracy: 0.8841
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.4293 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5562 - val_sparse_categorical_accuracy: 0.8190
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.3018 - sparse_categorical_accuracy: 0.9203
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.3006 - sparse_categorical_accuracy: 0.9190 - val_loss: 0.4944 - val_sparse_categorical_accuracy: 0.8490
Epoch 4/10
31/32 [=======

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [8]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.



In [9]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3620 - sparse_categorical_accuracy: 0.0990 - 243ms/epoch - 8ms/step
Untrained model, accuracy:  9.90%


Then load the weights from the checkpoint and re-evaluate:

In [10]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4028 - sparse_categorical_accuracy: 0.8630 - 218ms/epoch - 7ms/step
Restored model, accuracy: 86.30%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [11]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


Now, review the resulting checkpoints and choose the latest one:

In [12]:
os.listdir(checkpoint_dir)

['cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0035.ckpt.index',
 'checkpoint',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001']

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [15]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4892 - sparse_categorical_accuracy: 0.8750 - 202ms/epoch - 6ms/step
Restored model, accuracy: 87.50%


## Manually save weights

To save weights manually, use `tf.keras.Model.save_weights`. By default, `tf.keras`—and the `Model.save_weights` method in particular—uses the TensorFlow [Checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension.

In [16]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4892 - sparse_categorical_accuracy: 0.8750 - 233ms/epoch - 7ms/step
Restored model, accuracy: 87.50%


## Save the entire model

Call `tf.keras.Model.save` to save a model's architecture, weights, and training configuration in a single `model.keras` zip archive.

An entire model can be saved in three different file formats (the new `.keras` format and two legacy formats: `SavedModel`, and `HDF5`). Saving a model as `path/to/model.keras` automatically saves in the latest format.

**Note:** For Keras objects it's recommended to use the new high-level `.keras` format for richer, name-based saving and reloading, which is easier to debug. The low-level SavedModel format and legacy H5 format continue to be supported for existing code.

You can switch to the SavedModel format by:

- Passing `save_format='tf'` to `save()`
- Passing a filename without an extension

You can switch to the H5 format by:
- Passing `save_format='h5'` to `save()`
- Passing a filename that ends in `.h5`

### New high-level `.keras` format

The new Keras v3 saving format, marked by the `.keras` extension, is a more
simple, efficient format that implements name-based saving, ensuring what you load is exactly what you saved, from Python's perspective. This makes debugging much easier, and it is the recommended format for Keras.

The section below illustrates how to save and restore the model in the `.keras` format.

In [17]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 13ms/step - loss: 1.1569 - sparse_categorical_accuracy: 0.6840
Epoch 2/5
32/32 [==============================] - 0s 13ms/step - loss: 0.4226 - sparse_categorical_accuracy: 0.8770
Epoch 3/5
32/32 [==============================] - 0s 14ms/step - loss: 0.2860 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 14ms/step - loss: 0.2034 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 15ms/step - loss: 0.1577 - sparse_categorical_accuracy: 0.9640


Reload a fresh Keras model from the `.keras` zip archive:

In [18]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Try running evaluate and predict with the loaded model:

In [19]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 1s - loss: 0.4420 - sparse_categorical_accuracy: 0.8590 - 704ms/epoch - 22ms/step
Restored model, accuracy: 85.90%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The section below illustrates the steps to save and restore the model.

In [20]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 9ms/step - loss: 1.1926 - sparse_categorical_accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 9ms/step - loss: 0.4374 - sparse_categorical_accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 9ms/step - loss: 0.3007 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2068 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 10ms/step - loss: 0.1731 - sparse_categorical_accuracy: 0.9600


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [21]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


Reload a fresh Keras model from the saved model:

In [22]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [23]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4335 - sparse_categorical_accuracy: 0.8570 - 223ms/epoch - 7ms/step
Restored model, accuracy: 85.70%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


### HDF5 format

Keras provides a basic legacy high-level save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard.

In [24]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 9ms/step - loss: 1.1704 - sparse_categorical_accuracy: 0.6760
Epoch 2/5
32/32 [==============================] - 0s 9ms/step - loss: 0.4472 - sparse_categorical_accuracy: 0.8670
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2895 - sparse_categorical_accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2084 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 10ms/step - loss: 0.1556 - sparse_categorical_accuracy: 0.9690


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Now, recreate the model from that file:

In [25]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               401920    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Check its accuracy:

In [26]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4130 - sparse_categorical_accuracy: 0.8630 - 218ms/epoch - 7ms/step
Restored model, accuracy: 86.30%


Keras saves models by inspecting their architectures. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration (what you pass to the `.compile()` method)
* The optimizer and its state, if any (this enables you to restart training where you left off)

Keras is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.
